"""
This program analyses the trajectories made by participants using deocder G

(a) - without range control as we want to understand whether there is a null space in this decoder
"""

Start with importing the necessary libraries

In [ ]:
# Import libraries
import numpy as np
import matplotlib.pyplot as plt
import os
import pickle
import sys
from scipy import stats

# add current path to system PATH 
sys.path.insert(0,'/Users/ashwin/Documents/Y4 project Brain Human Interfaces/General 4th year Github repo/BrainMachineInterfaceDataAnalysis')

# Import user defined libraries
import DataExtraction.extractRawData as dataExtractor
from BasicAnalytics import targetAcqusitionPlotting as targetPlotter
from BasicAnalytics import variabilityAnalysis 
from BasicAnalytics import plottingFuncs


Fetch the gamesave data for decoder G, and invert the y axis of the positions

In [ ]:
# Fetch key decoding data for variability analysis rigid body data is shape: T x noDOF x noDecoders x noParticipants
rigidBodyDecoderData,positions, scores, noParticipants,informationTrialLists  = dataExtractor.retrieveDecoderData(version = 2)

labels = ['PVS', 'AB', 'CH', 'NCK', 'HD', 'LSD', 'LUA', 
                      'LFA', 'LHD', 'RSD', 'RUA', 'RFA', 'RHD',  'LTH', 'LSN', 
                      'LFT', 'RTH', 'RSN', 'RFT']

# Invert y axis as in game positive is downwards in y axis
positions[:,1,:,:] = 1025 - positions[:,1,:,:]

In [ ]:
# Define which participants used range control
rangeControl = [1,1,1,0,0,0,1,1,0,0,0]
usedRangeControl = [0,1,2,6,7]
noRangeControl = [3,4,5,8,9,10]

startingIdx = informationTrialLists[26]['startRigidBodyIdx']
calibrationMatrix = informationTrialLists[26]['calibrationMatrix']
rawRigidBodyData = informationTrialLists[26]['rawRigidBodyData']

In [ ]:
# Analyse range participants with no range control for now for left hand 
leftHandDecoderIdx = 2
leftHandRigidBodyMovements = rigidBodyDecoderData[:,:,leftHandDecoderIdx, noRangeControl].copy()
leftHandPositions = positions[:,:,leftHandDecoderIdx, noRangeControl].copy()



In [ ]:
leftHandPositions.shape

In [ ]:
# For no range control
plt.plot(leftHandPositions[:,0,:],leftHandPositions[:,1,:],label = [0,1,2,3,4,5])
plt.legend()
#plt.plot(leftHandPositions[:,0,0],leftHandPositions[:,1,0],label = [0])
plt.plot([0,1900],[0,0],color = 'r',linewidth = 2)
plt.plot([0,1900],[1025,1025], color = 'r',linewidth = 2)
plt.plot([0,0],[0,1025],color = 'r',linewidth = 2)
plt.plot([1900,1900],[0,1025],color = 'r',linewidth = 2)
plt.xlim(0,1900)
plt.ylim(0,1025)

# Retrieve decoder weights for left hand and look at where user moved left hand

#Look in training data where user could move left hand and whether 
coefficients, offsets, minsDOF, maxsDOF, DOFoffsets = dataExtractor.extractDecoderWeights(decoder = "G")

In [ ]:
# Debugging purposes only
# for idx,trial in enumerate(informationTrialLists):
#     print("trial no", idx)
#     print("cursorPos shape",trial['cursorPos'].shape)
#     print("cursorVel shape",trial['cursorVel'].shape)
#     print("rigidbodyData shape",trial['rigidBodyData'].shape)
coefficients[4]

In [ ]:
shared_array_copy = np.array([[ 0.11703367,  0.75340485,  0.93505347, -0.02130818,  0.02293149,
         0.99950999],
       [ 0.11535301,  0.75521358,  1.01388896, -0.30261619,  0.01977792,
         0.95290753],
       [ 0.05206556,  0.75934985,  1.21317482,  0.99622928,  0.0732663 ,
         0.04646855],
       [ 0.04046511,  0.77250051,  1.44114006,  0.05689667,  0.05088944,
         0.99708242],
       [ 0.06798203,  0.7774326 ,  1.58909237,  0.91412251, -0.34299986,
        -0.21617388],
       [ 0.03527027,  0.80706447,  1.38086593,  0.08388172,  0.99348696,
         0.07711996],
       [ 0.04258739,  0.89372747,  1.38759315,  0.15814788,  0.24096164,
        -0.95756299],
       [ 0.09400681,  0.97207252,  1.07625592,  0.34036135, -0.03070394,
        -0.93979337],
       [ 0.16698298,  0.96548936,  0.87475681,  0.2935919 ,  0.11928633,
        -0.9484592 ],
       [ 0.04068331,  0.73048469,  1.38555896,  0.15822079, -0.96723832,
         0.19853599],
       [ 0.0544851 ,  0.64611136,  1.40287745,  0.85408569, -0.47267666,
        -0.21705828],
       [ 0.33217821,  0.4924277 ,  1.33230424,  0.99637789, -0.00927247,
        -0.08452947],
       [ 0.54580952,  0.49043961,  1.31418049,  0.9773432 , -0.16077822,
        -0.13766073],
       [ 0.12277048,  0.84935967,  0.93297428,  0.99163619,  0.12453326,
         0.03389621],
       [ 0.13403827,  0.88047149,  0.48903045,  0.98268675,  0.13419261,
        -0.12774666],
       [ 0.07545336,  0.90161593,  0.06057939,  0.97281447,  0.22918343,
         0.03327516],
       [ 0.11129685,  0.65745003,  0.93713266,  0.96318066, -0.26877408,
         0.00659115],
       [ 0.1078196 ,  0.63408726,  0.49258432,  0.94663134, -0.27381835,
        -0.17003675],
       [ 0.03077431,  0.63229419,  0.06654397,  0.95779511, -0.28737914,
        -0.00647231]])

#xpos decode is array([[0.21903579]])
shared_array_copy.shape
shared_array_copy[8]

In [ ]:


# Process of calculating output
# 1. calibration - already performed for all rigid body data 
# 2. needs to be performed resiszing tmpRigBodyArray[DOF] =  (tmpRigBodyArray[DOF] - self.DOFmin[DOF]) / (self.DOFmax[DOF] - self.DOFmin[DOF] + self.DOFOffset) # very sensitive to the offset ???
# 3. output of model: self.xposDECODE = np.matmul(self.modelCoeff[0].reshape(1,-1),tmpArray.reshape(1,-1).transpose()) + self.modelIntercept[0]

# experiment on 500th data point on 3th participant
timestamp = 2000
length = 5000
participant = 4
positionDecoded = np.zeros((length,2))
DOFmax = maxsDOF[participant].copy()
DOFmin = minsDOF[participant].copy()
DOFOffset = DOFoffsets[participant].copy()
modelCoeff = coefficients[participant].copy()
modelIntercept = offsets[participant].copy()

rigidBodyArray = shared_array_copy.copy()
print('q',rigidBodyArray[8,:])
# Extract left hand rots
idxLeftHand = 8 * 6
rigidBodyArray = np.matmul(calibrationMatrix,rigidBodyArray.T).T.reshape(-1,1)
print("r",rigidBodyArray[48:54])
#rigidBodyArray[idxLeftHand+3:idxLeftHand+6] = np.matmul(calibrationMatrix[0:3,0:3],rigidBodyArray[idxLeftHand+3:idxLeftHand+6].reshape(3,1)).reshape(3)

for DOF in range(0,rigidBodyArray.shape[0]):
    rigidBodyArray[DOF] =  (rigidBodyArray[DOF] - DOFmin[DOF]) / (DOFmax[DOF] - DOFmin[DOF] + DOFOffset) # very sensitive to the offset ???
print(rigidBodyArray.shape)
print(rigidBodyArray[48:54])


tmpArray = rigidBodyArray[idxLeftHand:idxLeftHand+6][3:]
print("h",tmpArray)
print(DOFOffset)
# modelCoeff[0][1] = 0.7 * modelCoeff[0][1]
# modelCoeff[0][2] = 0.2 * modelCoeff[0][2]
# modelCoeff[0][0] = 0.2 * modelCoeff[0][0]
xposDECODE = np.matmul(modelCoeff[0].reshape(1,-1),tmpArray.reshape(1,-1).transpose()) + modelIntercept[0]
xPos = xposDECODE[0] * 1900
yposDECODE = np.matmul(modelCoeff[1].reshape(1,-1),tmpArray.reshape(1,-1).transpose()) + modelIntercept[1]
yPos = yposDECODE[0] * 1025

positionDecoded[timestamp,0] = xPos
positionDecoded[timestamp,1] = yPos

print(xPos,yPos,leftHandPositions[timestamp,:,2])


In [ ]:
#Look in training data where user could move left hand and whether 
coefficients, offsets, minsDOF, maxsDOF, DOFoffsets = dataExtractor.extractDecoderWeights(decoder = "G")

In [ ]:


# Process of calculating output
# 1. calibration - already performed for all rigid body data 
# 2. needs to be performed resiszing tmpRigBodyArray[DOF] =  (tmpRigBodyArray[DOF] - self.DOFmin[DOF]) / (self.DOFmax[DOF] - self.DOFmin[DOF] + self.DOFOffset) # very sensitive to the offset ???
# 3. output of model: self.xposDECODE = np.matmul(self.modelCoeff[0].reshape(1,-1),tmpArray.reshape(1,-1).transpose()) + self.modelIntercept[0]

# experiment on 500th data point on 3th participant
timestamp = 2000
length = 5000
participant = 4
rawRigidBodyData = informationTrialLists[22+participant]['rawRigidBodyData']
calibrationMatrix =  informationTrialLists[22+participant]['calibrationMatrix']
positionDecoded = np.zeros((length,2))
DOFmax = maxsDOF[participant].copy()
DOFmin = minsDOF[participant].copy()
DOFOffset = DOFoffsets[participant].copy()
modelCoeff = coefficients[participant].copy()
modelIntercept = offsets[participant].copy()
#calibrationMatrix = np.eye(6)


for timestamp in range(0,length): # length
    #rigidBodyArray = rigidBodyDecoderData[timestamp,:,2,participant].copy().reshape(-1,1)
    rigidBodyArray = rawRigidBodyData[timestamp,[0,1,2,3,4,5,6,7,8,24,25,26,27,43,44,45,47,48,49] ,:]
    # Extract left hand rots
    idxLeftHand = 8 * 6
    #rigidBodyArray[idxLeftHand+3:idxLeftHand+6] = np.matmul(calibrationMatrix[0:3,0:3],rigidBodyArray[idxLeftHand+3:idxLeftHand+6].reshape(3,1)).reshape(3)
    #rigidBodyArray[idxLeftHand+3:idxLeftHand+6] = np.matmul(calibrationMatrix[0:3,0:3],rigidBodyArray[idxLeftHand+3:idxLeftHand+6].reshape(3,1)).reshape(3)
    #rigidBodyArray = np.matmul(calibrationMatrix,rigidBodyArray.T).T
    rigidBodyArray = np.matmul(calibrationMatrix,rigidBodyArray.T).T.reshape(-1,1)
    print(rigidBodyArray[idxLeftHand:idxLeftHand+6][3:])
    for DOF in range(0,rigidBodyArray.shape[0]):
        # Very sensitive to the offset
        rigidBodyArray[DOF] =  ( rigidBodyArray[DOF] - DOFmin[DOF]) / (DOFmax[DOF] - DOFmin[DOF] + 0.1) # very sensitive to the offset ???


    


    
    tmpArray = rigidBodyArray[idxLeftHand:idxLeftHand+6][3:]
    
    # modelCoeff[0][1] = 0.7 * modelCoeff[0][1]
    # modelCoeff[0][2] = 0.2 * modelCoeff[0][2]
    # modelCoeff[0][0] = 0.2 * modelCoeff[0][0]
    xposDECODE = np.matmul(modelCoeff[0].reshape(1,-1),tmpArray.reshape(1,-1).transpose()) + modelIntercept[0]
    xPos = xposDECODE[0] * 1900
    yposDECODE = np.matmul(modelCoeff[1].reshape(1,-1),tmpArray.reshape(1,-1).transpose()) + modelIntercept[1]
    yPos = yposDECODE[0] * 1025
    print(xPos, yPos)
    positionDecoded[timestamp,0] = xPos
    positionDecoded[timestamp,1] = yPos

print(xPos,yPos,leftHandPositions[timestamp,:,2])

positionDecoded[:,1] = 1025 - positionDecoded[:,1]

plt.plot(positionDecoded[:,0],positionDecoded[:,1],label = "decoded")

plt.plot(positions[:length,0,2,participant],positions[:length,1,2,participant],label = "true")
plt.legend()
print(modelCoeff)
print(DOFOffset)
plt.xlim(0,1900)
plt.ylim(0,1025)
posTrues = positions[:length,:,2,participant].copy()

In [ ]:
import numpy as np

# Assuming X and Y are defined as N x 2 matrices
X = np.array(positionDecoded)  # Replace with your actual data
Y = np.array(posTrues)  # Replace with your actual data

# Augment X with a column of ones for the translation
X_augmented = np.hstack([X, np.ones((X.shape[0], 1))])

# Solve for the augmented A matrix, which includes the linear transformation and translation
A_augmented = np.linalg.lstsq(X_augmented, Y, rcond=None)[0]

# A_augmented is now a 3x2 matrix, where the first two columns are the linear transformation
# and the last column is the translation
A = A_augmented[:-1, :]
b = A_augmented[-1, :]


In [ ]:
positionsDecoded_recalc = np.matmul(A,X.T).T + b


In [ ]:
rigidBodyDecoderData[423,:,2,participant].copy()


In [ ]:
plt.plot(positionsDecoded_recalc[:,0],positionsDecoded_recalc[:,1],label = "decoded")
plt.plot(positions[:length,0,2,participant],positions[:length,1,2,participant],label = "true")
plt.legend()
print(modelCoeff)
print(DOFOffset)
posTrues = positions[:length,:,2,participant].copy()

In [ ]:
rigidBodyDecoderData[13,:,2,participant].copy()

In [ ]:
# Implementation of page 4

# Define u
u = np.zeros((19,51))
principalRigidBodyIndexes = [0,1,2,3,4,5,6,7,8,24,25,26,27,43,44,45,47,48,49]
for counter, principalRigidBodyIdx in enumerate(principalRigidBodyIndexes):
    u[counter,principalRigidBodyIdx] = 1

# Define D_lh
D_lh = np.zeros((3,114))
for i in range(3):
    D_lh[i,i+51] = 1
    

In [ ]:
# Extract trial data and decoder file
trialData = dataExtractor.processTrialData("ExperimentRuns/P5_Katsu_28_02/P5_Katsu_28_02__12_00_usingDecoderG")
decoderData = np.load("ExperimentRuns/P5_Katsu_28_02/P5_Katsu_28_02__12_00_linearRigidBodyGModel.npz")

# Extract trial data we need
pos = trialData['cursorPos']
rawRigidBodyData = trialData['rawRigidBodyData']
calibrationMatrix =  trialData['calibrationMatrix']

# Extract decoder data we need
DOFmax = decoderData["maxDOF"].copy()
DOFmin = decoderData["minDOF"].copy()
DOFOffset = decoderData["DOFOffset"].copy()
modelCoeff = decoderData["modelCoeff"].copy()
modelIntercept = decoderData["modelIntercept"].copy()



trainingData1 = dataExtractor.processTrialData("ExperimentRuns/P5_Katsu_28_02/P5_Katsu_28_02__12_00_training1")
calibrationTraining1Matrix =  trialData['calibrationMatrix']
training1RawRigidBodyData = trialData['rawRigidBodyData']
# training2RawRigidBodyData = dataExtractor.processTrialData("ExperimentRuns/P5_Katsu_28_02/P5_Katsu_28_02__12_00_training2")
# training3RawRigidBodyData = dataExtractor.processTrialData("ExperimentRuns/P5_Katsu_28_02/P5_Katsu_28_02__12_00_training3")
# training4RawRigidBodyData = dataExtractor.processTrialData("ExperimentRuns/P5_Katsu_28_02/P5_Katsu_28_02__12_00_training4")
# training5RawRigidBodyData = dataExtractor.processTrialData("ExperimentRuns/P5_Katsu_28_02/P5_Katsu_28_02__12_00_test")


In [ ]:
DOFOffset

In [ ]:
# Define other matrices

# Define C
C = calibrationMatrix

# Define d_min
d_min = np.matmul(D_lh,DOFmin)

# Define d_max
d_max = np.matmul(D_lh,DOFmax)

# Define alpha (DOFOffset)
alpha = 0.1

# Define D_l
d_l = 1 /(d_max - d_min + alpha)
D_l = np.zeros((3,3))
for i in range(3):
    D_l[i,i] = d_l[i]

# Define H
H = np.zeros((2,3))
H[0,:] = modelCoeff[0]
H[1,:] = modelCoeff[1]

# Define C
C_off = np.zeros(2)
C_off[0] = modelIntercept[0]
C_off[1] = modelIntercept[1]


In [ ]:
print(d_min)
print(d_max)
print(d_l)
print("(H D_l d_min)",np.matmul(H,np.matmul(D_l,d_min)))
print("(H D_l d_min)",H @ D_l @ d_min)
print("C", C_off )

# C + H D_l d_min
constTerms = C_off - np.matmul(H,np.matmul(D_l,d_min))
print("const",constTerms)

In [ ]:
print(D_l)


In [ ]:
# Implement matrix algorithm for calculation of decoded position
length = rawRigidBodyData.shape[0]
positionDecoded = np.zeros((length,2))

Y_hats = np.zeros((3,length))

for timestamp in range(0,length):
    
    # Retrieve raw data for current timestep
    X = rawRigidBodyData[timestamp,:,:].reshape(51,6)
    
    # Calculate Y_c the calibrated principle rigid body matrix
    Y_c = u @ X @ C.T
    
    # The unnormalised rigid body vector 
    Y_hat = D_lh @ Y_c.reshape(-1,1)
    Y_hats[:,timestamp] = Y_hat.reshape(3)

    # Calculate Z, the normalised rigid body vector
    Z = D_l @ (Y_hat.reshape(3,1) - d_min.reshape(3,1))

    # Calculate S, the unit length position
    S = H @ Z + C_off.reshape(2,1)

    # Store S
    positionDecoded[timestamp,0] = S[0] * 1900
    positionDecoded[timestamp,1] = S[1] * 1025



# Plot decoded and actual
plt.plot(positionDecoded[:,0],1025 - positionDecoded[:,1],label = "decoded")
plt.plot(pos[:,0],1025 - pos[:,1],label = "true")
plt.legend()
plt.xlim(0,1900)
plt.ylim(0,1025)


In [ ]:
# We have S = A * Y_hat + Off
# both elements of S need to be between 0 - 1

# First let's take an SVD of A to find the directions
A = H @ D_l

# find eigenvectors and eigenvalues
U,S,Vh = np.linalg.svd(A)
print(S)
print(U)
# Reverse distances 
U[0][1] = -U[0][1]# * 1900/1025
U[1][1] = -U[1][1] #* 1900/1025
# Plot the vector
norm = S[0] * U[0] + S[1] * U[1]
norm[0] = norm[0] 

plt.plot(positionDecoded[:,0],1025 - positionDecoded[:,1],label = "decoded",alpha = 0.5)
plt.plot(pos[:,0],1025 - pos[:,1],label = "true",alpha = 0.5)
plt.quiver(750,700,- U[0][0], -U[0][1],scale = 20/(S[0]), color='blue')
plt.quiver(750,700, -U[1][0], -U[1][1],scale = 20/(S[1]), color='red')
plt.quiver(750,700, -norm[0], -norm[1],scale = 20/(S[1]),width = 0.02, color='green')
plt.legend()
# plt.xlim(0,1900)
# plt.ylim(0,1025)
print(A)


In [ ]:
print(Vh)

In [ ]:
#Now let's perform constrained optimisation to understand what areas we can move in

d_range = np.zeros((3,50))
for i in range(3):
    d_range[i,:] = np.linspace(d_min[0],d_max[0],50)
# yhat [[ 0.2935919 ]
#  [ 0.11928633]
#  [-0.9484592 ]]
# s [[0.47195332]
#  [0.48383055]]
y_test = np.array([ 0.2935919, 0.11928633,-0.9484592])
S = A @ y_test.reshape(3,1) + constTerms.reshape(2,1)
print(S)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Step 2: Define the ranges for x, y, and z
x_range = (d_min[0], d_max[0])
y_range = (d_min[1], d_max[1])
z_range = (d_min[2], d_max[2])

# Step 3: Create a grid of x, y, z points
x = np.linspace(-1, 1, 100)
y = np.linspace(-1, 1, 100)
z = np.linspace(-1, 1, 100)
x, y, z = np.meshgrid(x, y, z)

# Step 4: Apply the constraints
inside = (x >= x_range[0]) & (x <= x_range[1]) & \
         (y >= y_range[0]) & (y <= y_range[1]) & \
         (z >= z_range[0]) & (z <= z_range[1])

# Step 5: Visualization
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(x[inside], y[inside], z[inside], c='r', marker='o')
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.set_title('Visualizing a Constrained Volume in 3D Space')
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Step 2: Define the ranges for x, y, and z
# Step 2: Define the ranges for x, y, and z
delta = 0.1
d_min_ = np.min(Y_hats,axis = 1)
d_max_ = np.max(Y_hats,axis = 1)
d_min_ = d_min
d_max_ = d_max
x_range = (d_min_[0] - delta ,d_max_[0] + delta)
y_range = (d_min_[1] - delta, d_max_[1] + delta)
z_range = (d_min_[2] - delta, d_max_[2] + delta)


# Step 3: Create a grid of x, y, z points and apply the transformation
x = np.linspace(x_range[0], x_range[1], 40)
y = np.linspace(y_range[0], y_range[1], 40)
z = np.linspace(z_range[0], z_range[1], 40)
# x = np.ones(20)* 0.2935919
# y = np.ones(20) * 0.11928633
# z = np.ones(20) * -0.9484592
x, y, z = np.meshgrid(x, y, z)

# Flatten the arrays to make them suitable for matrix multiplication
points_3d = np.vstack([x.ravel(), y.ravel(), z.ravel()])
print(points_3d.shape)
#points_3d = Y_hats

# Apply the transformation matrix to project the 3D points to 2D
points_2d = A @ points_3d + constTerms.reshape(2,1)
print(points_2d.shape)
# Step 4: Visualization
plt.figure(figsize=(6, 6))
plt.scatter(1900*points_2d[0], 1025 - 1025*points_2d[1], color='blue', s=1)  # Scatter plot of 2D points
plt.title('Projected 2D Area from 3D Volume')
plt.xlabel('Projected X')
plt.ylabel('Projected Y')
plt.plot(positionDecoded[:,0],1025 - positionDecoded[:,1],label = "decoded",alpha = 0.5)
plt.plot(pos[:,0],1025 - pos[:,1],label = "true",alpha = 0.5)
plt.quiver(750,700,- U[0][0], -U[0][1],scale = 20/(S[0]), color='blue')
plt.quiver(750,700, -U[1][0], -U[1][1],scale = 20/(S[1]), color='red')
plt.quiver(750,700, -norm[0], -norm[1],scale = 20/(S[1]),width = 0.02, color='green')
plt.legend()
#plt.xlim(0,1900)
#plt.ylim(0,1025)
plt.grid(True)
plt.show()


In [ ]:
print(np.min(Y_hats,axis = 1))
print(np.max(Y_hats,axis = 1))
print(d_min)
print(d_max)

In [ ]:
trainingData1 = dataExtractor.processTrialData("ExperimentRuns/P5_Katsu_28_02/P5_Katsu_28_02__12_00_test")
calibrationTraining1Matrix =  trainingData1['calibrationMatrix']
training1RawRigidBodyData = trainingData1['rawRigidBodyData']


# Calculate Y_c
Y_c_training1 = np.tensordot( u , np.tensordot(training1RawRigidBodyData,calibrationTraining1Matrix.T, axes = [2,0]) ,axes = [1,1])
print("l",Y_c_training1.shape)

# FINE UP TO HERE

# Form Y_hat
# 19 x 9000 x 6
Y_c_training1 = np.transpose(Y_c_training1, (0, 2, 1))
# plt.plot(Y_c_training1[8,:,3])
Y_c_training1 = Y_c_training1.reshape(114,-1,order = 'C')
print("l2",Y_c_training1.shape)
#plt.plot(Y_c_training1[51,:])
Y_hat_training1 = np.tensordot(D_lh,Y_c_training1, axes = [1,0])[:,:5500]
print(Y_hat_training1.shape)
print(np.min(Y_hat_training1,axis=1))
print(np.max(Y_hat_training1,axis=1))
print(d_min)
print(d_max)

plt.plot(Y_hat_training1[:,:5500].T)
plt.show()
plt.plot(Y_hats[:,:5500].T)
print(Y_hat_training1[:,0].shape)
print(np.max(Y_hat_training1[:,:]))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Step 2: Define the ranges for x, y, and z
# Step 2: Define the ranges for x, y, and z
delta = 1
# d_min_ = np.min(Y_hat_training1,axis = 1)
# d_max_ = np.max(Y_hat_training1,axis = 1)
d_min_ = d_min
d_max_ = d_max
x_range = (d_min_[0] - delta ,d_max_[0] + delta)
y_range = (d_min_[1] - delta, d_max_[1] + delta)
z_range = (d_min_[2] - delta, d_max_[2] + delta)


# Step 3: Create a grid of x, y, z points and apply the transformation
x = np.linspace(x_range[0], x_range[1], 40)
y = np.linspace(y_range[0], y_range[1], 40)
z = np.linspace(z_range[0], z_range[1], 40)
# x = np.ones(20)* 0.2935919
# y = np.ones(20) * 0.11928633
# z = np.ones(20) * -0.9484592
x, y, z = np.meshgrid(x, y, z)

# Flatten the arrays to make them suitable for matrix multiplication
points_3d = np.vstack([x.ravel(), y.ravel(), z.ravel()])
print(points_3d.shape)
#points_3d = Y_hats

# Apply the transformation matrix to project the 3D points to 2D
points_2d = A @ Y_hat_training1 + constTerms.reshape(2,1)
print(points_2d.shape)
# Step 4: Visualization
plt.figure(figsize=(6, 6))
plt.scatter(1900*points_2d[0], 1025 - 1025*points_2d[1], color='blue', s=1)  # Scatter plot of 2D points
# plt.title('Projected 2D Area from 3D Volume')
# plt.xlabel('Projected X')
# plt.ylabel('Projected Y')
# plt.plot(positionDecoded[:,0],1025 - positionDecoded[:,1],label = "decoded",alpha = 0.5)
# plt.plot(pos[:,0],1025 - pos[:,1],label = "true",alpha = 0.5)
# plt.quiver(750,700,- U[0][0], -U[0][1],scale = 20/(S[0]), color='blue')
# plt.quiver(750,700, -U[1][0], -U[1][1],scale = 20/(S[1]), color='red')
# plt.quiver(750,700, -norm[0], -norm[1],scale = 20/(S[1]),width = 0.02, color='green')
plt.legend()
plt.xlim(0,1900)
plt.ylim(0,1025)
plt.grid(True)
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Step 2: Define the ranges for x, y, and z
# Step 2: Define the ranges for x, y, and z
delta = 1
# d_min_ = np.min(Y_hat_training1,axis = 1)
# d_max_ = np.max(Y_hat_training1,axis = 1)
d_min_ = d_min
d_max_ = d_max
x_range = (d_min_[0] - delta ,d_max_[0] + delta)
y_range = (d_min_[1] - delta, d_max_[1] + delta)
z_range = (d_min_[2] - delta, d_max_[2] + delta)


# Step 3: Create a grid of x, y, z points and apply the transformation
x = np.linspace(x_range[0], x_range[1], 40)
y = np.linspace(y_range[0], y_range[1], 40)
z = np.linspace(z_range[0], z_range[1], 40)
# x = np.ones(20)* 0.2935919
# y = np.ones(20) * 0.11928633
# z = np.ones(20) * -0.9484592
x, y, z = np.meshgrid(x, y, z)

# Flatten the arrays to make them suitable for matrix multiplication
points_3d = np.vstack([x.ravel(), y.ravel(), z.ravel()])
print(points_3d.shape)
#points_3d = Y_hats

# Apply the transformation matrix to project the 3D points to 2D
points_2d = A @ Y_hats[:,:1000] + constTerms.reshape(2,1)
print(points_2d.shape)
# Step 4: Visualization
plt.figure(figsize=(6, 6))
plt.scatter(1900*points_2d[0], 1025 - 1025*points_2d[1], color='blue', s=1)  # Scatter plot of 2D points
# plt.title('Projected 2D Area from 3D Volume')
# plt.xlabel('Projected X')
# plt.ylabel('Projected Y')
# plt.plot(positionDecoded[:,0],1025 - positionDecoded[:,1],label = "decoded",alpha = 0.5)
# plt.plot(pos[:,0],1025 - pos[:,1],label = "true",alpha = 0.5)
# plt.quiver(750,700,- U[0][0], -U[0][1],scale = 20/(S[0]), color='blue')
# plt.quiver(750,700, -U[1][0], -U[1][1],scale = 20/(S[1]), color='red')
# plt.quiver(750,700, -norm[0], -norm[1],scale = 20/(S[1]),width = 0.02, color='green')
plt.legend()
# plt.xlim(0,1900)
# plt.ylim(0,1025)
plt.grid(True)
plt.show()


In [ ]:
print(Vh) # 1st dir in largest variation - predominantly third component